# Amazon Transcribe Audio Toxicity Detection

Amazon Transcribe now provides a simple, ML based solution for toxic content detection in audio chats. The capability available for social media, gaming, and general needs, eliminating the requirement for customers to bring their own data for training the ML model. The toxicity detection feature leverages the context of the conversation and also audio cues(e.g., pitch, tone, emotion) to detect toxic content.

In this tutorial we will learn how to detect toxicity in audio files. 
You can learn more [here](https://aws.amazon.com/transcribe/toxicity-detection/).

![audio-moderation-arch](../images/audio-moderation-transcribe-toxicity-detection.png)

- [Step 1: Setup Notebook](#step1)
- [Step 2: Setup Variables and Import Packages](#step2)
- [Step 3: Setup Input Audio File & Run Amazon Transcribe Toxicity Detection Job](#step3)
- [Step 4: Check Toxicity Detection Results](#step4)

# Step 1: Setup Notebook <a id="step1"></a>
Run the below cell to install/update Python dependencies. 

In [ ]:
# install the below packages
%pip install -qU pip
%pip install -qU boto3
%pip install -qU json2html

# Step 2: Setup Variables and Import Packages <a id="step2"></a>

In [ ]:
import boto3
import sagemaker as sm
import os
import datetime
import time
import json

# variables
data_bucket = sm.Session().default_bucket()
region = boto3.session.Session().region_name

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sm.get_execution_role()
#The role should have SagemakerFullAccess and TranscribeFullAccess
print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3')
transcribe_client=boto3.client('transcribe', region_name=region)

# Step 3: Setup input audio file & Run Amzon Transcribe Toxicity Detection Job  <a id="step3"></a>
Run the below cell to upload a sample audio file (mp3) to the default S3 bucket for Transcribe to access.

In [ ]:
#Here we use a file from the datasets directory and upload it to S3. You can modify the name/path for s3_key, location of audio file to upload. 
#Also change the name of job_name if you are rerunning the job.
s3_key = 'content-moderation-im/audio-moderation/moderation-audio-speech.mp3'
s3.upload_file('../datasets/moderation-audio-speech.mp3', data_bucket, s3_key)
file_uri = 's3://'+data_bucket+'/'+s3_key
print(file_uri)

Call Transcribe **StartTranscriptionJob** API and set ToxicityDetection parameter to transcribe the audio to text and detect toxic content.You should also set language to "en-US". Currently only English language is supported for this feature. Amazon Transcribe StartTranscriptionJob is an asynchronous API that will start a job managed by Transcribe. We will then call the **GetTranscriptionJob** API to check the job status until the job completes.

In [ ]:
import uuid
job_name = f'toxicity_detection_{str(uuid.uuid1())[0:4]}'

transcribe_client.start_transcription_job(
        TranscriptionJobName = job_name,
        Media = {
            'MediaFileUri': file_uri
        },
        OutputBucketName = data_bucket,
        OutputKey = 'content-moderation-im/audio-moderation/my-output-files/',
        MediaFormat = 'mp3',
        LanguageCode = 'en-US',
        ToxicityDetection = [{'ToxicityCategories': ['ALL']}]
    )
max_tries = 60
while max_tries > 0:
    max_tries -= 1
    job = transcribe_client.get_transcription_job(TranscriptionJobName = job_name)
    job_status = job['TranscriptionJob']['TranscriptionJobStatus']
    if job_status in ['COMPLETED', 'FAILED']:
        print(f"Job {job_name} is {job_status}.")
        if job_status == 'COMPLETED':
            print(
                f"transcription output file copied to\n"
                f"\t{job['TranscriptionJob']['Transcript']['TranscriptFileUri']}.")
        break
    else:
        print(f"Waiting for {job_name}. Current status is {job_status}.")
    time.sleep(10)

The job is completed. Transcribe stored the toxicity detection output data to a JSON file in the S3 path specified in the `OutputKey` parameter. We now open the transcribed output JSON file from S3 and check the transcription accuracy. Notice that some of the words may be inappropriate.

# Step 4: Check Toxicity Detection Results <a id="step4"></a>
Amazon Transcribe allows you to detect different types of toxic content and also provides a confidence score between 0 and 1 for each category. The supported cateories for toxicity deection includes:

* Profanity: Speech that contains words, phrases, or acronyms that are impolite, vulgar, or oﬀensive.
* Hate speech: Speech that criticizes, insults, denounces, or dehumanizes a person or group on the basis of an identity (such as race, ethnicity, gender, religion, sexual orientation, ability, and national origin).
* Sexual: Speech that indicates sexual interest, activity, or arousal using direct or indirect references to body parts, physical traits, or sex.
* Insults: Speech that includes demeaning, humiliating, mocking, insulting, or belittling language. This type of language is also labeled as bullying.
* Violence or threat: Speech that includes threats seeking to inﬂict pain, injury, or hostility toward a person or group.
* Graphic: Speech that uses visually descriptive and unpleasantly vivid imagery. This type of language is often intentionally verbose to amplify a recipient's discomfort.
* Harassment or abusive: Speech intended to aﬀect the psychological well-being of the recipient, including demeaning and objectifying terms. 

In [ ]:
#Now, let's show reasults in a more readable format.
from json2html import *
from IPython.display import HTML

filename = 'content-moderation-im/audio-moderation/my-output-files/'+job_name+'.json'
s3_clientobj = s3.get_object(Bucket=data_bucket, Key=filename)
s3_clientdata = s3_clientobj["Body"].read().decode("utf-8")
original = json.loads(s3_clientdata)
output_transcript = original["results"].get("toxicity_detection")
HTML(json2html.convert(json = output_transcript))

# Cleanup: <a id="cleanup"></a>
We will delete the audio file in the S3 bucket and the Audio Filter in Transcribe to clean up resources and prevent unnecessary costs.

In [ ]:
s3.delete_object(Bucket=data_bucket, Key=s3_key)

# Conclusion: <a id="conclusion"></a>
In this lab, we learned how to use Toxocity Detection feature with Amazon transcribe for audio moderation.